In [2]:
from copy import deepcopy
from pprint import pprint
from random import randint, choice, random, shuffle
from typing import List

# =============================
# Hyperparameters
# =============================

CITIES_NUMBER = 10
INDIVIDUALS_NUMBER = 40
SELECTED_PARENT = int(INDIVIDUALS_NUMBER / 2)
MUTATION_PROBA = 0.07



In [3]:
# =============================
# UTILS
# =============================


def strip_population(population: List) -> List:
    assert isinstance(population[0], List)
    population_striped = [individual[0] for individual in population]
    return population_striped


def remove_duplicate_individual(population: List) -> List:
    new_population = []

    for individual in population:
        not_exit = True
        for selected_indv in new_population:
            if selected_indv == individual:
                not_exit = False
                break
        if not_exit:
            new_population.append(individual)

    return new_population


def validate_city_revisited(individual: List) -> bool:
    indiv_set = set(individual)
    return len(individual) == len(indiv_set)


def validate_visit_all_cities_once(individual: List) -> bool:
    if not validate_city_revisited(individual):
        return False
    return len(individual) == CITIES_NUMBER



In [4]:
# ============================
# GA
# ============================


def generate_cities(number_of_citiies: int):
    cities = []
    for i in range(number_of_citiies):
        city = []
        for j in range(number_of_citiies):
            if i == j:
                city.append(0)
            else:
                city.append(randint(1, 100))
        cities.append(city)

    for i in range(number_of_citiies):
        for j in range(number_of_citiies):
            cities[i][j] = cities[j][i]

    return cities



cities = generate_cities(10)
pprint(cities)

[[0, 71, 61, 73, 19, 48, 63, 52, 32, 79],
 [71, 0, 70, 65, 32, 27, 97, 3, 29, 65],
 [61, 70, 0, 4, 59, 21, 1, 94, 84, 37],
 [73, 65, 4, 0, 28, 47, 50, 93, 17, 98],
 [19, 32, 59, 28, 0, 54, 91, 11, 87, 61],
 [48, 27, 21, 47, 54, 0, 52, 11, 11, 76],
 [63, 97, 1, 50, 91, 52, 0, 34, 92, 4],
 [52, 3, 94, 93, 11, 11, 34, 0, 11, 54],
 [32, 29, 84, 17, 87, 11, 92, 11, 0, 91],
 [79, 65, 37, 98, 61, 76, 4, 54, 91, 0]]


In [5]:
# ============================
# Initialization
# ============================


def generate_individuale(start_node: int = None):
    if not start_node:
        start_node = randint(0, CITIES_NUMBER - 1)

    cities: List = list(range(CITIES_NUMBER))
    shuffle(cities)
    return cities


def generate_first_population(individuals_number):
    population = []
    for i in range(individuals_number):
        population.append(generate_individuale())

    return population

population = generate_first_population(10)
pprint(population)

[[2, 8, 6, 0, 7, 9, 4, 1, 5, 3],
 [1, 4, 7, 5, 3, 2, 0, 8, 9, 6],
 [1, 3, 5, 0, 4, 7, 6, 8, 2, 9],
 [4, 7, 8, 9, 1, 0, 5, 2, 6, 3],
 [3, 9, 5, 7, 0, 1, 4, 2, 6, 8],
 [2, 7, 0, 6, 8, 3, 1, 5, 9, 4],
 [5, 1, 8, 6, 2, 9, 4, 3, 7, 0],
 [7, 5, 1, 3, 6, 2, 0, 8, 4, 9],
 [6, 8, 4, 5, 3, 9, 7, 0, 1, 2],
 [0, 4, 2, 1, 6, 9, 5, 7, 3, 8]]


In [6]:
# ============================
# Evaluation
# ============================


def path_total_distance(cities, path):
    score = 0
    for i in range(len(path) - 1):
        city_1 = path[i]
        city_2 = path[i + 1]
        score += cities[city_1][city_2]

    return score


def fitness_reverse(cities, path) -> int:
    return path_total_distance(cities, path) * -1


def fitness_reverse_devide(cities, path) -> float:
    return 1 / path_total_distance(cities, path)


def evaulate_population(cities, population):
    evaluation = []
    for i in range(len(population)):
        indivudal = population[i]
        evaluation.append([indivudal, fitness_reverse_devide(cities, indivudal)])

    return evaluation


population_evaluated = evaulate_population(cities, population)
pprint(population_evaluated)

[[[2, 8, 6, 0, 7, 9, 4, 1, 5, 3], 0.001953125],
 [[1, 4, 7, 5, 3, 2, 0, 8, 9, 6], 0.0034129692832764505],
 [[1, 3, 5, 0, 4, 7, 6, 8, 2, 9], 0.002288329519450801],
 [[4, 7, 8, 9, 1, 0, 5, 2, 6, 3], 0.0027100271002710027],
 [[3, 9, 5, 7, 0, 1, 4, 2, 6, 8], 0.0020325203252032522],
 [[2, 7, 0, 6, 8, 3, 1, 5, 9, 4], 0.0018281535648994515],
 [[5, 1, 8, 6, 2, 9, 4, 3, 7, 0], 0.002380952380952381],
 [[7, 5, 1, 3, 6, 2, 0, 8, 4, 9], 0.002531645569620253],
 [[6, 8, 4, 5, 3, 9, 7, 0, 1, 2], 0.0016],
 [[0, 4, 2, 1, 6, 9, 5, 7, 3, 8], 0.002242152466367713]]


In [7]:

# ============================
# Selection
# ============================


# sort method
def second_element(elem):
    return elem[1]


def order_population(population: List):
    population_copy = population.copy()
    population_copy.sort(key=second_element, reverse=True)
    return population_copy


def get_best_individual(individual_list: List):
    individual_list.sort(key=second_element, reverse=True)
    return individual_list[:INDIVIDUALS_NUMBER]


def calc_relative_portion(population: List):
    population_with_portion = deepcopy(population)
    sum_of_score = 0

    for individual in population_with_portion:
        sum_of_score += individual[1]

    for individual in population_with_portion:
        prec = individual[1] / sum_of_score

        if len(individual) >= 3:
            individual[2] = prec
        else:
            individual.append(prec)

    return population_with_portion


def select_by_probability(population_with_portion, proba):
    some = 0
    for index, individual in enumerate(population_with_portion):
        some += individual[2]
        if proba < some:
            return index

    raise Exception(f"Proba out of range some={some}, proba={proba}")


def roulette_wheel_selection(population: List):
    population_used = population.copy()
    selected_parent = []
    for i in range(SELECTED_PARENT):
        population_with_portion = calc_relative_portion(population_used)
        random_value = random()
        index_selected_parent = select_by_probability(
            population_with_portion, random_value
        )
        selected_individual = population_used.pop(index_selected_parent)
        selected_parent.append(selected_individual)

    return selected_parent

p = calc_relative_portion(population_evaluated)
#pprint(p)

In [8]:
# ============================
# Croisement
# ============================


def generate_new_chromosome(
    individual_1: List, individual_2: int, start_index: int, end_index: int
):
    assert len(individual_1) == len(individual_2)

    individual_length = len(individual_1)

    new_chromosome = [0] * len(individual_1)

    value_used = []

    # remplace first futures
    for index in range(start_index, end_index):
        new_chromosome[index] = individual_2[index]
        value_used.append(individual_2[index])

# ============================
# Selection
# ============================

# sort method
def second_element(elem):
    return elem[1]


def order_population(population: List):
    population_copy = population.copy()
    population_copy.sort(key=second_element, reverse=True)
    return population_copy


def get_best_individual(individual_list: List):
    individual_list.sort(key=second_element, reverse=True)
    return individual_list[:INDIVIDUALS_NUMBER]


def calc_relative_portion(population: List):
    population_with_portion = deepcopy(population)
    sum_of_score = 0

    for individual in population_with_portion:
        sum_of_score += individual[1]

    for individual in population_with_portion:
        prec = individual[1] / sum_of_score

        if len(individual) >= 3:
            individual[2] = prec
        else:
            individual.append(prec)

    return population_with_portion


def select_by_probability(population_with_portion, proba):
    some = 0
    for index, individual in enumerate(population_with_portion):
        some += individual[2]
        if proba < some:
            return index

    raise Exception(f"Proba out of range some={some}, proba={proba}")


def roulette_wheel_selection(population: List):
    population_used = population.copy()
    selected_parent = []
    for i in range(SELECTED_PARENT):
        population_with_portion = calc_relative_portion(population_used)
        random_value = random()
        index_selected_parent = select_by_probability(
            population_with_portion, random_value
        )
        selected_individual = population_used.pop(index_selected_parent)
        selected_parent.append(selected_individual)

    return selected_parent

    # apply order crossover
    current_selected_index = end_index
    current_selected_value = end_index

    counter = 0

    while counter < individual_length:
        if individual_1[current_selected_value] not in value_used:
            new_chromosome[current_selected_index] = individual_1[
                current_selected_value
            ]
            value_used.append(individual_1[current_selected_value])
            current_selected_index += 1

        current_selected_value += 1
        current_selected_index = current_selected_index % individual_length
        current_selected_value = current_selected_value % individual_length

        counter += 1

    return new_chromosome


def ordered_crossover(individual_1: List, individual_2: List) -> tuple:
    assert len(individual_1) == len(individual_2)

    cut_length = 2

    start_cut_point = 2
    end_cut_point = start_cut_point + cut_length

    new_chromosome_1 = generate_new_chromosome(
        individual_1, individual_2, start_cut_point, end_cut_point
    )

    new_chromosome_2 = generate_new_chromosome(
        individual_2, individual_1, start_cut_point, end_cut_point
    )

    return (new_chromosome_1, new_chromosome_2)


def random_crossover_selection(parent: List, number_of_crossover: int):
    generated_children = []
    for i in range(number_of_crossover):
        shuffle(parent)
        selected_parent_1, selected_parent_2 = parent[:2]
        children = ordered_crossover(selected_parent_1, selected_parent_2)
        generated_children.extend(children)

    return generated_children


In [9]:
# ============================
# Mutation
# ============================

def individual_mutation(indv: List):
    gene_1 = randint(0, len(indv) - 1)
    gene_2 = randint(0, len(indv) - 1)
    indv[gene_2], indv[gene_1] = indv[gene_1], indv[gene_2]


def population_mutation(population, mutation_proba):
    assert 1 >= mutation_proba >= 0
    population_length = len(population)
    individual_length = len(population[0])
    mutation_ratio = round((population_length * individual_length) * mutation_proba)
    children = deepcopy(population)

    for i in range(mutation_ratio):
        selected_parent = choice(children)
        individual_mutation(selected_parent)

    return children


In [10]:

cities = generate_cities(CITIES_NUMBER)
population = generate_first_population(INDIVIDUALS_NUMBER)
current_individuals = evaulate_population(cities, population)


for i in range(100):
    selected_parent = roulette_wheel_selection(current_individuals)
    striped_parent = strip_population(selected_parent)
    population_mutated = population_mutation(striped_parent, 0.5)

    population.extend(population_mutated)

    valid_indvidual = remove_duplicate_individual(population)

    evaluate_population = evaulate_population(cities, valid_indvidual)
    best_individual = get_best_individual(evaluate_population)
    current_individuals = best_individual

print("best indvidual is ")
print(best_individual[0][0])


best indvidual is 
[6, 4, 9, 1, 5, 8, 7, 2, 0, 3]
